반복적으로 사용되는 프롬프트를 템플릿으로 만들자

In [ ]:
# -q 플래그는 설치 중에 출력되는 메시지를 최소화합니다.
# openai: OpenAI API와 상호작용하기 위한 라이브러리
# langchain: 다양한 언어 모델과의 연결 및 데이터 파이프라인을 구축하기 위한 라이브러리
# langchain-openai: LangChain과 OpenAI API를 통합하는 기능을 제공하는 추가 라이브러리
!pip install -q openai langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00


In [ ]:
!pip install -q langchain_community # 다양한 추가 기능 및 도구를 제공하여 LangChain의 활용도를 높입니다.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os
#채팅 LLM 로드를 위한 라이브러리 호출
from langchain.chat_models import ChatOpenAI

In [ ]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = ## API Key ##

-------

#### 모델 설정하기

In [ ]:
chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens = 512)

<ipython-input-6-53add3f9e9ca>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens = 512)


#### (1) 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")  # subject를 중괄호로 넣어서 가변적인 매개변수로 설정

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer")  # subject에 어떤것을 넣을지 설정

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

In [ ]:
#to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능
print(string_prompt_value.to_string())

tell me a joke about soccer


In [ ]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}") # 이것도 위와 같은 기능
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer', additional_kwargs={}, response_metadata={})])

In [ ]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

In [ ]:
answer = chatgpt(chat_prompt_value.to_messages())
print(answer.content)

Why was the soccer field so hot? Because all the fans left!


#### (2) 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
from langchain.prompts.prompt import PromptTemplate
# 템플릿 정의: 요리사 역할을 부여하고, 사용 가능한 재료를 입력받아 요리를 추천하고 레시피를 제시하는 프롬프트
template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
# PromptTemplate 객체 생성: 템플릿과 입력 변수를 지정하여 프롬프트 템플릿을 만듭니다
prompt_template = PromptTemplate(
    input_variables = ['재료'],  # 리스트 형태로 사용 (여러개를 사용할 수 있음)
    template = template
)


In [ ]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵')) # prompt_template의 format() 메서드를 사용하여 '재료' 변수에 '양파, 계란, 사과, 빵' 값을 할당하고, 최종 프롬프트를 생성하여 출력합니다.


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [ ]:
from langchain.schema import HumanMessage # langchain 라이브러리에서 HumanMessage 클래스를 임포트합니다. HumanMessage는 LLM에게 전달할 메시지를 담는 객체입니다.
# HumanMessage 객체를 생성하여 LLM에게 전달할 메시지를 설정
answer = chatgpt(
    [HumanMessage(content=prompt_template.format(
        재료 = '양파, 계란, 사과, 빵'
    ))]
)
print(answer.content)

네, 제가 추천하는 요리는 "양파 계란말이" 입니다. 

<양파 계란말이 레시피>
1. 양파를 얇게 채썰어 준비합니다.
2. 계란 2개를 풀어 볼에 넣고 소금으로 간을 해줍니다.
3. 팬에 식용유를 두르고 양파를 볶다가 계란을 부어줍니다.
4. 계란이 익을 때쯤 양념을 해주고 섞어줍니다.
5. 부드럽게 익은 양파 계란말이를 빵과 함께 내어 드세요.

재료로 가지고 있는 양파와 계란으로 간단하게 만들 수 있는 요리이니 맛있게 즐기시길 바랍니다. 만약 추가적인 요리나 레시피가 필요하시다면 말씀해주세요!


##  ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
# chatgpt의 프롬프트 템플릿을 더 잘 활용하기 위해서는 ChatPromptTemplate을 사용
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# ChatGPT 모델을 로드합니다.
chatgpt = ChatOpenAI(temperature=0)

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""

#ChatGPT에게 역할을 부여합니다.(위에서 정의한 Template 사용)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

#사용자가 입력할 매개변수 template을 선언합니다.
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿을 삽입합니다.
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#ChatGPT API에 ChatPromptTemplate을 입력할 때, human message의 매개변수인 '재료'를 할당하여 전달합니다.
#이와 같은 방식을 통해 ChatGPT는 ChatPromptTemplate의 구성요소인 system message와 human message를 전달받아, 대답 생성에 활용합니다.
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

제가 추천하는 요리는 "양파 계란 토스트"입니다. 이 요리는 간단하면서도 맛있는 아침 식사나 간식으로 딱 좋아요. 준비물과 레시피는 아래와 같습니다.

<양파 계란 토스트>

<재료>
- 빵 4조각
- 계란 2개
- 양파 1개
- 소금
- 후추
- 식용유

<레시피>
1. 양파를 얇게 슬라이스해줍니다.
2. 팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶아줍니다.
3. 볶은 양파를 접시에 옮겨놓고 팬을 세척하지 않습니다.
4. 계란 2개에 소금과 후추를 넣고 푹푹 섞어줍니다.
5. 팬에 계란을 부어 반숙 계란을 만들어줍니다.
6. 계란이 익으면 빵을 넣어 한쪽 면이 바삭해질 때까지 구워줍니다.
7. 빵을 뒤집어 반대편도 구워줍니다.
8. 빵 한 쪽에 양파를 얹고, 그 위에 계란을 올려줍니다.
9. 다른 빵으로 덮어주고 완성합니다.

맛있는 양파 계란 토스트가 완성되었어요. 맛있게 드세요!


###(3) Few-shot 예제를 통한 프롬프트 템플릿

Few-shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론입니다.

LLM 역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력합니다.

내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있습니다.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "호날두로 삼행시 만들어줘",
    "answer":
"""
호: 호날두는 축구장에서
날: 날라다니며
두: 두발로 골을 넣는다
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0])) # **는 Python에서 키워드 인수 언팩킹
                                            # 이를 통해 딕셔너리의 키-값 쌍을 함수에 인수로 전달
                                            # examples[0]의 내용을 question과 answer라는 변수로 각각 전달

Question: 호날두로 삼행시 만들어줘

호: 호날두는 축구장에서
날: 날라다니며
두: 두발로 골을 넣는다



In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}", # 사용자 입력(input)이 추가될 부분을 지정합니다. "Question: {input}" 형태이며, 사용자의 질문이 {input}에 들어갑니다.
    input_variables=["input"]  # 입력 변수로 "input"을 사용합니다.
)

print(prompt.format(input="손흥민으로 삼행시 만들어줘"))

Question: 호날두로 삼행시 만들어줘

호: 호날두는 축구장에서
날: 날라다니며
두: 두발로 골을 넣는다


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 손흥민으로 삼행시 만들어줘


In [ ]:
print(chatgpt.invoke("손흥민으로 삼행시 만들어줘")) #퓨샷 템플릿 사용을 안했을 경우

content='손흥민은 축구 신동\n공을 차면 상대팀 무너져\n슈팅 강하고 빠른 발끝' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 26, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-445deb42-6853-4fc0-a396-84e193dc6564-0'


In [ ]:
answer = chatgpt([HumanMessage(content=prompt.format(input="손흥민으로 삼행시 만들어줘"))])
print(answer.content)

손: 손흥민은 토트넘에서
흥: 흥미진진한 경기를 펼친다
민: 민낯으로 팬들을 사로잡는다
